# Lab 5 - Using link functions to create generalized linear models

In [ ]:
from cmdstanpy import CmdStanModel
import arviz as az
import numpy as np
import scipy.stats as stats
import scipy.stats as norm
import matplotlib.pyplot as plt
import pandas as pd

dark="#EECA02"
dark_highlight="#BB9700"

In [ ]:
# _BASE_URL = "https://raw.githubusercontent.com/KAIR-ISZ/lectures/main/Data%20Analytics/Lab%205/Airline_data.csv"
# AIRPLANE_DATASET_PATH = f"{_BASE_URL}/Airline_data.csv"
# df = pd.read_csv(AIRPLANE_DATASET_PATH, sep=';', header=0)
# df.head()

data = pd.read_csv('https://raw.githubusercontent.com/KAIR-ISZ/lectures/main/Data%20Analytics/Lab%205/Airline_data.csv')
data.head()
display(data)
print(data.describe())


## Model for accidents
Our considered model were (assuming y<sub>i</sub>  is the number of accidents in given year):

    
$$ y_i \sim \mathrm{Poisson}(\lambda) $$

that is one fixed &lambda; for all years i.e. number of accidents is random independent on year where it happens

### Prior predictive distribution

In [ ]:
model_prior=CmdStanModel(stan_file='airline_FA_ppc.stan')

ppc = model_prior.sample(data=dict(N=len(data), 
                         Miles_flown=data["Miles flown [100 mln miles]"].values))
ppc.summary()

In [ ]:
ppc_pd = ppc.draws_pd()
fig = plt.figure(figsize=(10, 7))

ppc_pd["y_mean"] = ppc_pd[[f"y[{i}]" 
                           for i in range(1, 11)]].mean(axis=1)
ppc_pd["y_mean"].hist(bins='auto', color=dark,edgecolor=dark_highlight,linewidth=2)
plt.bar(data["Fatal accidents"], 
        [50 for _ in range(data["Fatal accidents"].size)], color="red")
plt.show()


## Modelling for accidents, considering miles
It is rather logical, that number of accidents should be related to number of miles flown. We considered model were (assuming y<sub>i</sub>  is the number of accidents in given year):
$$ y_i \sim \mathrm{Poisson}(\theta* n_i) $$

where n<sub>i</sub> is the ammount of miles flown that year.